In [1]:
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras import layers
from sklearn.metrics import plot_confusion_matrix, recall_score, accuracy_score, f1_score, make_scorer, precision_score
from time import perf_counter 
import seaborn as sns
import os

In [2]:
## Defining batch specfications
batch_size = 100
img_height = 224
img_width = 224

In [3]:
# loading training set
training_ds = tf.keras.preprocessing.image_dataset_from_directory(
    './dataset/train',
    seed=42,
    image_size= (img_height, img_width),
    batch_size=batch_size

)

Found 69079 files belonging to 37 classes.


In [4]:
# loading training set
validation_ds = tf.keras.preprocessing.image_dataset_from_directory(
    './dataset/valid/',
    seed=42,
    image_size= (img_height, img_width),
    batch_size=batch_size

)

Found 17068 files belonging to 37 classes.


In [5]:
class_names = training_ds.class_names
print(class_names)

['Apple___Black_rot', 'Apple___Cedar_apple_rust', 'Apple___healthy', 'Blueberry___healthy', 'Cherry___Powdery_mildew', 'Cherry___healthy', 'Corn___Cercospora_leaf_spot Gray_leaf_spot', 'Corn___Common_rust_', 'Corn___Northern_Leaf_Blight', 'Corn___healthy', 'Grape___Black_rot', 'Grape___Esca', 'Grape___Leaf_blight', 'Grape___healthy', 'Orange___Haunglongbing', 'Peach___Bacterial_spot', 'Peach___healthy', 'Pepperbell___Bacterial_spot', 'Pepperbell___healthy', 'Potato___Early_blight', 'Potato___Late_blight', 'Potato___healthy', 'Raspberry___healthy', 'Soybean___healthy', 'Squash___Powdery_mildew', 'Strawberry___Leaf_scorch', 'Strawberry___healthy', 'Tomato___Bacterial_spot', 'Tomato___Early_blight', 'Tomato___Late_blight', 'Tomato___Leaf_Mold', 'Tomato___Septoria_leaf_spot', 'Tomato___Spider_mites Two-spotted_spider_mite', 'Tomato___Target_Spot', 'Tomato___Tomato_Yellow_Leaf_Curl_Virus', 'Tomato___Tomato_mosaic_virus', 'Tomato___healthy']


In [10]:
# Download the mobilenet model from MobileNetV2
model_mobile = tf.keras.applications.MobileNetV2(
    input_shape=(224,224,3),
    alpha=1.0,
    include_top=False,
    weights="imagenet",
    input_tensor=None,
    classes=37,
    classifier_activation="softmax",
)

In [11]:
model_mobile.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [12]:
model_mobile.fit(training_ds, validation_data = validation_ds, batch_size = 32, epochs = 1)

InvalidArgumentError:  Incompatible shapes: [100,7,7] vs. [100,1]
	 [[node Equal (defined at <ipython-input-12-144873ca34fc>:1) ]] [Op:__inference_train_function_19833]

Function call stack:
train_function


In [35]:
#https://arxiv.org/abs/1704.04861

In [47]:
#Freeze the training of the model                                         
model_mobile.trainable = False

# Create new model on top
inputs = tf.keras.Input(shape=(224, 224, 3))
input_x = modeltest(inputs, training=False)
input_x = tf.keras.layers.GlobalAveragePooling2D()(input_x)
input_x = tf.keras.layers.Dropout(0.2)(input_x)  
outputs = tf.keras.layers.Dense((38),activation="softmax")(input_x)

model = tf.keras.Model(inputs, outputs)
model.fit(training_ds, validation_data = validation_ds, batch_size = 32, epochs = 1)

ValueError: Input 0 of layer dense_15 is incompatible with the layer: expected axis -1 of input shape to have value 107648 but received input with shape (None, 86528)

In [17]:
image_path = './dataset/train/Apple___Black_rot/0090d05d-d797-4c99-abd4-3b9cb323a5fd___JR_FrgE.S 8727_270deg.JPG'

In [20]:
from tensorflow.keras.preprocessing.image import load_img, img_to_array, array_to_img
import numpy as np

image = load_img(image_path,target_size=(224,224))
image = img_to_array(image)
image = image/255
image = np.expand_dims(image,axis=0)
result = model.predict(image)
result1 = np.argmax(result)
np.round(result,10000)
print(result)
print(result1)

[[0.02730026 0.01645253 0.05648718 0.00770764 0.01229987 0.03482609
  0.02640763 0.02311678 0.00117793 0.00674621 0.03419627 0.0744663
  0.00130962 0.01920535 0.05094594 0.0051055  0.08573176 0.01317156
  0.00780189 0.00387529 0.02080188 0.0287405  0.02821501 0.01423869
  0.03954937 0.09437226 0.00019041 0.01073176 0.01459055 0.0043519
  0.00828294 0.00492792 0.04737097 0.02200865 0.04328324 0.00852823
  0.00775913 0.09372502]]
25


In [21]:
modeltest = tf.keras.models.load_model('../leaf-model.h5') #UNCOMMENT TO LOAD MODEL

In [22]:
from tensorflow.keras.preprocessing.image import load_img, img_to_array, array_to_img
import numpy as np

image = load_img(image_path,target_size=(224,224))
image = img_to_array(image)
image = image/255
image = np.expand_dims(image,axis=0)
result = modeltest.predict(image)
result1 = np.argmax(result)
np.round(result,10000)
print(result)
print(result1)

[[3.9190672e-05 8.7886781e-01 2.6092031e-03 1.4233454e-04 2.1424721e-05
  1.6666038e-04 4.7422682e-06 6.4253601e-07 1.7963710e-06 2.6573721e-06
  9.5821431e-09 9.7588190e-06 7.6653751e-06 2.2443815e-05 1.1893886e-05
  8.9615044e-09 5.5007313e-05 1.3267027e-02 1.6518835e-02 3.7412567e-06
  1.5444155e-06 1.8918343e-05 2.2444064e-04 6.1982867e-08 1.4183892e-07
  1.9113993e-06 1.1886928e-08 5.1043876e-09 3.1450821e-07 1.6698007e-03
  2.6889492e-03 3.5103183e-04 1.4942091e-02 9.0368273e-04 6.6913284e-02
  4.6925089e-07 4.6475031e-04 6.5940447e-05]]
1


C:\Users\jesse\Anaconda3\lib\site-packages\numpy\core\fromnumeric.py:58: RuntimeWarning: invalid value encountered in true_divide
  return bound(*args, **kwds)


In [ ]:
model.predict()

In [ ]:
tf.keras.applications.MobileNetV2(
    input_shape=None,
    alpha=1.0,
    include_top=True,
    weights="imagenet",
    input_tensor=None,
    pooling=None,
    classes=1000,
    classifier_activation="softmax",
    **kwargs
)

In [ ]:
####################################################################################################

In [ ]:
mobile = keras.applications.mobilenet.MobileNet()

In [27]:
from keras.layers import Dense,GlobalAveragePooling2D
from keras.models import Model
from keras.preprocessing.image import ImageDataGenerator
from keras.applications.mobilenet import preprocess_input

In [31]:
base_model=tf.keras.applications.MobileNetV2(weights='imagenet',
                                             include_top=False,
                                            input_shape=(224,224,3)) #imports the mobilenet model and discards the last 1000 neuron layer.

x=base_model.output
x=GlobalAveragePooling2D()(x)
x=Dense(1024,activation='relu')(x) #we add dense layers so that the model can learn more complex functions and classify for better results.
x=Dense(1024,activation='relu')(x) #dense layer 2
x=Dense(512,activation='relu')(x) #dense layer 3
preds=Dense(37,activation='softmax')(x) #final layer with softmax activation

In [32]:
model=Model(inputs=base_model.input,outputs=preds)
#specify the inputs
#specify the outputs
#now a model has been created based on our architecture

In [33]:
for i,layer in enumerate(model.layers):
  print(i,layer.name)

0 input_6
1 Conv1
2 bn_Conv1
3 Conv1_relu
4 expanded_conv_depthwise
5 expanded_conv_depthwise_BN
6 expanded_conv_depthwise_relu
7 expanded_conv_project
8 expanded_conv_project_BN
9 block_1_expand
10 block_1_expand_BN
11 block_1_expand_relu
12 block_1_pad
13 block_1_depthwise
14 block_1_depthwise_BN
15 block_1_depthwise_relu
16 block_1_project
17 block_1_project_BN
18 block_2_expand
19 block_2_expand_BN
20 block_2_expand_relu
21 block_2_depthwise
22 block_2_depthwise_BN
23 block_2_depthwise_relu
24 block_2_project
25 block_2_project_BN
26 block_2_add
27 block_3_expand
28 block_3_expand_BN
29 block_3_expand_relu
30 block_3_pad
31 block_3_depthwise
32 block_3_depthwise_BN
33 block_3_depthwise_relu
34 block_3_project
35 block_3_project_BN
36 block_4_expand
37 block_4_expand_BN
38 block_4_expand_relu
39 block_4_depthwise
40 block_4_depthwise_BN
41 block_4_depthwise_relu
42 block_4_project
43 block_4_project_BN
44 block_4_add
45 block_5_expand
46 block_5_expand_BN
47 block_5_expand_relu
48 b

In [34]:
for layer in model.layers:
    layer.trainable=False
# or if we want to set the first 20 layers of the network to be non-trainable
# for layer in model.layers[:20]:
#     layer.trainable=False
# for layer in model.layers[20:]:
#     layer.trainable=True

In [35]:
train_datagen=ImageDataGenerator(preprocessing_function=preprocess_input) #included in our dependencies

train_generator=train_datagen.flow_from_directory('./dataset/train',
                                                 target_size=(224,224),
                                                 color_mode='rgb',
                                                 batch_size=32,
                                                 class_mode='categorical',
                                                 shuffle=True)

Found 69079 images belonging to 37 classes.


In [ ]:
model.compile(optimizer='Adam',loss='categorical_crossentropy',metrics=['accuracy'])
# Adam optimizer
# loss function will be categorical cross entropy
# evaluation metric will be accuracy

In [43]:
step_size_train=train_generator.n//train_generator.batch_size
model.fit_generator(generator=train_generator,
                   steps_per_epoch=step_size_train,
                   epochs=4)

Epoch 1/4
2158/2158 [==============================] - 9269s 4s/step - loss: 0.2135 - accuracy: 0.9437
Epoch 2/4
2158/2158 [==============================] - 9169s 4s/step - loss: 0.1624 - accuracy: 0.9581
Epoch 3/4
2158/2158 [==============================] - 7860s 4s/step - loss: 0.1334 - accuracy: 0.9646
Epoch 4/4
2158/2158 [==============================] - 7584s 4s/step - loss: 0.1201 - accuracy: 0.9678


In [44]:
model.save('./models/leaf-model00.h5')

In [45]:
model.save('./models/my_model007')

INFO:tensorflow:Assets written to: ./models/my_model007\assets


In [92]:
from tensorflow.keras.preprocessing.image import load_img, img_to_array, array_to_img

image_path = './dataset/train/Cherry___healthy/0008f3d3-2f85-4973-be9a-1b520b8b59fc___JR_HL 4092.JPG'

image = load_img(image_path,target_size=(224,224))
image = img_to_array(image)
image = image/255
image = np.expand_dims(image,axis=0)
result = model.predict(image)
result1 = np.argmax(result)
print(result)
print(result1)

[[4.3279743e-03 1.3296792e-04 3.5011359e-02 1.5593014e-02 5.9411972e-04
  2.3772854e-01 8.5142805e-05 5.3746926e-06 2.7821237e-05 9.9041485e-05
  7.9981942e-04 9.2554124e-05 1.3895749e-04 1.6968938e-02 1.7017912e-03
  2.7972207e-04 2.1328842e-03 1.0373111e-02 3.3609748e-01 5.7728670e-04
  3.3297235e-04 3.0129794e-02 2.1456394e-03 2.9178578e-01 2.3294502e-04
  4.6956085e-04 4.3278709e-03 2.6347025e-03 1.1842053e-04 2.4892058e-04
  7.1870221e-05 2.9064849e-04 8.8532142e-05 7.4420089e-04 1.4130145e-03
  2.9056975e-05 2.1680358e-03]]
18


In [90]:
# Check its architecture
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_6 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
Conv1 (Conv2D)                  (None, 112, 112, 32) 864         input_6[0][0]                    
__________________________________________________________________________________________________
bn_Conv1 (BatchNormalization)   (None, 112, 112, 32) 128         Conv1[0][0]                      
__________________________________________________________________________________________________
Conv1_relu (ReLU)               (None, 112, 112, 32) 0           bn_Conv1[0][0]                   
____________________________________________________________________________________________

In [91]:
modeltest = tf.keras.models.load_model('../leaf-model.h5') #UNCOMMENT TO LOAD MODEL
modeltest.summary()

Model: "leaf_disease_model_mobilenet"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_4 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
mobilenet_1.00_224 (Function (None, 7, 7, 1024)        3228864   
_________________________________________________________________
global_average_pooling2d_1 ( (None, 1024)              0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 1024)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 38)                38950     
Total params: 3,267,814
Trainable params: 38,950
Non-trainable params: 3,228,864
_________________________________________________________________
